In [8]:
import qiskit
from qiskit.circuit import QuantumCircuit
from qiskit.providers.aer.noise import NoiseModel
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd
from mitiq import Executor, MeasurementResult, Observable
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq
import mitiq
from mitiq import zne, benchmarks, pec
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise




In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)


In [15]:

import qiskit_aer.noise as noise

# Error probabilities
prob_1 = 0.001  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['rz', '', 'x'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
print(noise_model)
print(noise_model.noise_instructions)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['', 'rz', 'cx', 'x']
  All-qubits errors: ['rz', '', 'x', 'cx']
['', 'cx', 'rz', 'x']


In [10]:
noise_dict = pd.read_pickle('./NoiseModel/fakekolkata.pkl')
# Load the noise model from the dictionary
noise_model = NoiseModel.from_dict(noise_dict)
print(noise_model)

/tmp/ipykernel_584123/1734049327.py:3: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(noise_dict)


NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['sx', '', 'measure', 'reset', 'id', 'x', 'cx']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2,)), ('id', (3,)), ('id', (4,)), ('id', (5,)), ('id', (6,)), ('id', (7,)), ('id', (8,)), ('id', (9,)), ('id', (10,)), ('id', (11,)), ('id', (12,)), ('id', (13,)), ('id', (14,)), ('id', (15,)), ('id', (16,)), ('id', (17,)), ('id', (18,)), ('id', (19,)), ('id', (20,)), ('id', (21,)), ('id', (22,)), ('id', (23,)), ('id', (24,)), ('id', (25,)), ('id', (26,)), ('sx', (0,)), ('sx', (1,)), ('sx', (2,)), ('sx', (3,)), ('sx', (4,)), ('sx', (5,)), ('sx', (6,)), ('sx', (7,)), ('sx', (8,)), ('sx', (9,)), ('sx', (10,)), ('sx', (11,)), ('sx', (12,)), ('sx', (13,)), ('sx', (14,)), ('sx', (15,)), ('sx', (16,)), ('sx', (17,)), ('sx', (18,)), ('sx', (19,)), ('sx', (20,)), ('sx', (21,)), ('sx', 

In [4]:
def execute_with_qiskit_noise(
    circuit: QuantumCircuit,
    noise_model: NoiseModel,
    shots=1,
    ideal = False
):
    qc = circuit.copy()
    qc.save_density_matrix()
    basis_gates = noise_model.basis_gates + ["save_density_matrix"]
    if(ideal):
        job = qiskit.execute(
            qc,
            backend=AerSimulator(),
            optimization_level=0,
            shots=shots
        )
    else:
        job = qiskit.execute(
            qc,
            backend=AerSimulator(method = "density_matrix", noise_model = noise_model),
            basis_gates=basis_gates,
            optimization_level=0,
            shots=shots
        )
    rho = job.result().data()["density_matrix"]
    return rho[0,0].real
    

In [5]:
qc = QuantumCircuit(12)

for i in range(qc.num_qubits):
    qc.rx(np.pi/2,i)
    qc.rx(-np.pi/2,i)
ideal_value = execute_with_qiskit_noise(qc,noise_model,1,True)
noisy_value = execute_with_qiskit_noise(qc,noise_model,1,False)

print(ideal_value)
print(noisy_value)

/tmp/ipykernel_3299780/2825760794.py:26: DeprecationWarning: The return type of saved density matrices has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.DensityMatrix` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  return rho[0,0].real


1.0
0.9837828954117973


In [6]:
noise_level_1 = 0.01
reps_1 = represent_operations_in_circuit_with_local_depolarizing_noise(qc,noise_level_1)
print(f"{len(reps_1)} OperationRepresentation_1 objects produced, assuming {100 * noise_level_1}% depolarizing noise.")

pec_value_1 = pec.execute_with_pec(qc,execute_with_qiskit_noise(qc,noise_model,1,False),representations=reps_1)
print(f"Error without PEC:                     {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with PEC(correct noise level):   {abs(ideal_value - pec_value_1) :.5f}")

24 OperationRepresentation_1 objects produced, assuming 1.0% depolarizing noise.


/tmp/ipykernel_3299780/2825760794.py:26: DeprecationWarning: The return type of saved density matrices has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.DensityMatrix` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  return rho[0,0].real


TypeError: unsupported callable

In [4]:
from mitiq.pec.representations import local_depolarizing_kraus
from mitiq.pec.channels import kraus_to_super
import qiskit
from qiskit.circuit import QuantumCircuit
from qiskit.providers.aer.noise import NoiseModel
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd
from mitiq import Executor, MeasurementResult, Observable
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq
import mitiq
from mitiq import zne, benchmarks, pec
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise


basis_circuit_h = qiskit.QuantumCircuit(1)
basis_circuit_h.h(0)

basis_circuit_hx = qiskit.QuantumCircuit(1)
basis_circuit_hx.h(0)
basis_circuit_hx.x(0)

basis_circuit_hy = qiskit.QuantumCircuit(1)
basis_circuit_hy.h(0)
basis_circuit_hy.y(0)

basis_circuit_hz = qiskit.QuantumCircuit(1)
basis_circuit_hz.h(0)
basis_circuit_hz.z(0)

basis_circuits = [basis_circuit_h, basis_circuit_hx, basis_circuit_hy, basis_circuit_hz] 

for c in basis_circuits:
    print(c)
    
    
# Compute depolarizing superoperator
BASE_NOISE = 0.2
depo_super = kraus_to_super(local_depolarizing_kraus(BASE_NOISE, num_qubits=1))

# Define the superoperator matrix of each noisy operation
super_matrices = [
    depo_super @ kraus_to_super([qiskit.quantum_info.Operator(c).data]) 
    for c in basis_circuits
] 

# Define NoisyOperation objects combining circuits with channel matrices
noisy_operations = [
    pec.NoisyOperation(circuit=c, channel_matrix=m)
    for c, m in zip(basis_circuits, super_matrices)
]

print(f"{len(noisy_operations)} NoisyOperation objects defined.")
print(noisy_operations)
print(super_matrices)

   ┌───┐
q: ┤ H ├
   └───┘
   ┌───┐┌───┐
q: ┤ H ├┤ X ├
   └───┘└───┘
   ┌───┐┌───┐
q: ┤ H ├┤ Y ├
   └───┘└───┘
   ┌───┐┌───┐
q: ┤ H ├┤ Z ├
   └───┘└───┘
4 NoisyOperation objects defined.
[<mitiq.pec.types.types.NoisyOperation object at 0x7f4f5026f9a0>, <mitiq.pec.types.types.NoisyOperation object at 0x7f4e658bae30>, <mitiq.pec.types.types.NoisyOperation object at 0x7f4e67937a30>, <mitiq.pec.types.types.NoisyOperation object at 0x7f4e6588e3b0>]
[array([[ 0.5       +0.j,  0.36666667+0.j,  0.36666667+0.j,
         0.5       +0.j],
       [ 0.36666667+0.j, -0.36666667+0.j,  0.36666667+0.j,
        -0.36666667+0.j],
       [ 0.36666667+0.j,  0.36666667+0.j, -0.36666667+0.j,
        -0.36666667+0.j],
       [ 0.5       +0.j, -0.36666667+0.j, -0.36666667+0.j,
         0.5       +0.j]]), array([[ 0.5       +0.j, -0.36666667+0.j, -0.36666667+0.j,
         0.5       +0.j],
       [ 0.36666667+0.j,  0.36666667+0.j, -0.36666667+0.j,
        -0.36666667+0.j],
       [ 0.36666667+0.j, -0.36666667+0.

In [3]:
import numpy as np
def standard_gate_unitary(name):
    # To be removed with from_dict
    unitary_matrices = {
        ("id", "I"): np.eye(2, dtype=complex),
        ("x", "X"): np.array([[0, 1], [1, 0]], dtype=complex),
        ("y", "Y"): np.array([[0, -1j], [1j, 0]], dtype=complex),
        ("z", "Z"): np.array([[1, 0], [0, -1]], dtype=complex),
        ("h", "H"): np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2),
        ("s", "S"): np.array([[1, 0], [0, 1j]], dtype=complex),
        ("sdg", "Sdg"): np.array([[1, 0], [0, -1j]], dtype=complex),
        ("t", "T"): np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]], dtype=complex),
        ("tdg", "Tdg"): np.array([[1, 0], [0, np.exp(-1j * np.pi / 4)]], dtype=complex),
        ("cx", "CX", "cx_01"): np.array(
            [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]], dtype=complex
        ),
        ("cx_10",): np.array(
            [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]], dtype=complex
        ),
        ("cz", "CZ"): np.array(
            [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]], dtype=complex
        ),
        ("swap", "SWAP"): np.array(
            [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]], dtype=complex
        ),
        ("ccx", "CCX", "ccx_012", "ccx_102"): np.array(
            [
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
            ],
            dtype=complex,
        ),
        ("ccx_021", "ccx_201"): np.array(
            [
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
            ],
            dtype=complex,
        ),
        ("ccx_120", "ccx_210"): np.array(
            [
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 1, 0],
            ],
            dtype=complex,
        ),
    }

    return next((value for key, value in unitary_matrices.items() if name in key), None)

In [11]:
a = []
a.append(standard_gate_unitary('abc')) if (standard_gate_unitary('abc') != None)
a.append('1')
a
a[0]==None

True

In [4]:
from qiskit.circuit.library import RZGate

print(RZGate(0.1).to_matrix())

[[0.99875026-0.04997917j 0.        +0.j        ]
 [0.        +0.j         0.99875026+0.04997917j]]
